<center>
    <b><h1> Couserwork Assignment: Text Classification</h1></b>
</center>

<center>
    <h3>University of London</h3>
</center>

<center>
    <h3>BSc in Computer Science</h3>
</center>

<center>
    <h3>Natural Language Processing</h3>
</center>

<div style="page-break-after: always;"></div>

<h1> Table of Content</h1>

<style>
ul {
    font-size: 20px;
}

ul ul {
    font-size: 15px;
}
</style>

<ul>
    <li>
        <a href="#intro">1. Introduction</a>
        <ul>
            <li><a href="#1.1">1.1 Problem Area</a></li>
            <li><a href="#1.2">1.2 Objectives</a></li>
        </ul>
    </li>
    <li>
        <a href="#section2">Section 2</a>
        <ul>
            <li><a href="#subsection2.1">Subsection 2.1</a></li>
            <li><a href="#subsection2.2">Subsection 2.2</a></li>
        </ul>
    </li>
    <li>
        <a href="#section3">Section 3</a>
        <ul>
            <li><a href="#subsection3.1">Subsection 3.1</a></li>
            <li><a href="#subsection3.2">Subsection 3.2</a></li>
        </ul>
    </li>
</ul>


<div style="page-break-after: always;"></div>

<h2 id="intro">1. Introduction</h2>
<h3 id="1.1">1.1 Problem area</h3>

<h3 id="">Data Extraction</h3>

In [1]:
# Import library needed for data analysis
import os
import random
import pandas as pd

In [2]:
# Path to the folders
pos_folder = r'C:\Users\xjie\Documents\SIM\Y3S2\nlp\aclImdb\train\pos'
neg_folder = r'C:\Users\xjie\Documents\SIM\Y3S2\nlp\aclImdb\train\neg'

In [3]:
# Helper functions;

# Count the number of files in the data folder
def count_files(folder_path):
    if not os.path.isdir(folder_path):
        print(f"Error: {folder_path} is not a valid directory.")
        return

    file_count = 0

    for _, _, files in os.walk(folder_path):
        file_count += len(files)

    return file_count

In [4]:
# Example the positive training data file
file_count = count_files(pos_folder)
print(f"The number of files in positive training folder is: {file_count}")

The number of files in positive training folder is: 12500


In [5]:
# Example the positive training data file
file_count = count_files(pos_folder)
print(f"The number of files in positive training folder is: {file_count}")

The number of files in positive training folder is: 12500


As the training data set is <b>too big</b> for the computer, only <b>2000 data</b> from positive and negative folder will be selected to make the program runs faster.

In [6]:
# Helper functions
# Function to extract unique ID and rating from file name
def extract_info(file_name):
    id_rating = file_name.split('.')[0]
    unique_id, rating = id_rating.split('_')
    return int(unique_id), int(rating)

# Function to extract the data from the file based on the aclImdb data structure
def extract_files(folder, num_files, sentiment):
    files = random.sample(os.listdir(folder), num_files)
    data = pd.DataFrame(columns=['Sentiment', 'Unique ID', 'Rating', 'Content'])
    
    for file_name in files:
        file_path = os.path.join(folder, file_name)
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read() # read content for the review
        unique_id, rating = extract_info(file_name)  # get ID and rating from file name
        file_data = pd.DataFrame({'Sentiment': [sentiment], 'Unique ID': [unique_id], 'Rating': [rating], 'Content': [content]})
        data = pd.concat([data, file_data], ignore_index=True)
    
    return data

In [7]:
# Extracting training data
pos_data = extract_files(pos_folder, 2000, 'pos')
neg_data = extract_files(neg_folder, 2000, 'neg')

# Print the resulting DataFrame
raw_data = pd.concat([pos_data, neg_data], ignore_index=True)

# Display the start of the data
raw_data.head()

,Sentiment,Unique ID,Rating,Content
0,pos,3325,7,"OK, I would not normally watch a Farrelly brot..."
1,pos,8233,8,Fido is a story about more well mannered zombi...
2,pos,7214,7,Ira Levin's Deathtrap is one of those mystery ...
3,pos,5069,7,The movie is more about Pony than Grey Owl. It...
4,pos,69,10,I was up late flipping cable channels one nigh...


<h3 id="">Prelimary analysis and data cleaning</h3>

In [8]:
# Examining the data
raw_data.describe()

,Sentiment,Unique ID,Rating,Content
count,4000,4000,4000,4000
unique,2,3683,8,3995
top,pos,7480,1,"I was one of those ""few Americans"" that grew u..."
freq,2000,2,801,2


It was noted that both Unique ID and Content has duplicated values. Thus, an algorithm will be implemented to check for these values and handle them.

In [9]:
# Helper function
# Check for repetetion of data
def find_duplicates(dataframe, column):
    duplicates = dataframe[dataframe[column].duplicated(keep=False)]
    duplicates_sorted = duplicates.sort_values(by=column)

    if duplicates_sorted.empty:
        print("No repeated data entry found.")
    else:
        print("Repeated data entry/entries found in "+str(column))
        duplicates_pairs = duplicates_sorted.groupby(column).apply(lambda x: x.reset_index(drop=True))
        duplicates_pairs.reset_index(drop=True, inplace=True)
        return duplicates_pairs

In [10]:
duplicate_content = find_duplicates(raw_data, 'Content')
duplicate_ID = find_duplicates(raw_data, 'Unique ID')

if duplicate_content is not None:
    print('-' * 50)
    print("Duplication in content:")
    print(duplicate_content)
    print('-' * 50)
if duplicate_ID is not None:
    print('-' * 50)
    print("Duplication in ID:")
    print(duplicate_ID)
    print('-' * 50)

Repeated data entry/entries found in Content
Repeated data entry/entries found in Unique ID
--------------------------------------------------
Duplication in content:
  Sentiment Unique ID Rating   
0       neg     10332      1  \
1       neg       281      1   
2       neg      7355      4   
3       neg     12216      4   
4       neg      2156      1   
5       neg     11575      1   
6       neg      5984      4   
7       neg      5983      4   
8       neg      5952      4   
9       neg      5550      4   

                                             Content  
0  Awful, simply awful. It proves my theory about...  
1  Awful, simply awful. It proves my theory about...  
2  Don't get me wrong - I love David Suchet as Po...  
3  Don't get me wrong - I love David Suchet as Po...  
4  I have been familiar with the fantastic book o...  
5  I have been familiar with the fantastic book o...  
6  I was one of those "few Americans" that grew u...  
7  I was one of those "few Americans" th

As a <b>new ID is assigned</b> as the index for the data frame, there is <b>no need</b> for the Unique ID from positive sentiment and negative sentiment folder. Thus, the <b>unique ID column will be dropped</b>. 

Moreover, the duplicated content will be <b>remove<b/> and new data will be loaded to replace them so that the dataset remains balanced

In [11]:
# Remove unique ID column from the data
org_ID_removed_data = raw_data.drop("Unique ID", axis=1)

In [12]:
# replace duplicates with other values
def get_balance_data_with_duplicates(duplicate_content, original_data, pos_folder, neg_folder):
    # Remove the first occurrence of unique items in the duplicated content
    data_to_remove = duplicate_content.drop_duplicates(subset='Content', keep='first')
    org_ID_removed_data = original_data.drop_duplicates(subset='Content', keep='first')

    # Count the number of pos and neg entries in 'data_to_remove'
    pos_count = data_to_remove[data_to_remove['Sentiment'] == 'pos'].shape[0]
    neg_count = data_to_remove[data_to_remove['Sentiment'] == 'neg'].shape[0]

    # Add the removed duplicates such that the data is balanced
    added_pos_data = extract_files(pos_folder, pos_count, 'pos')
    added_neg_data = extract_files(neg_folder, neg_count, 'neg')

    # Remove the 'Unique ID' column from added data
    added_pos_data.drop("Unique ID", axis=1, inplace=True)
    added_neg_data.drop("Unique ID", axis=1, inplace=True)

    added_data = pd.concat([added_pos_data, added_neg_data], ignore_index=True)
    balanced_data = pd.concat([org_ID_removed_data, added_data], ignore_index=True)

    return balanced_data

In [13]:
def recursive_data_balancing(dataframe, column, pos_folder, neg_folder):
    duplicates = find_duplicates(dataframe, column)

    if duplicates is None:
        print("No more duplicates found.")
        return dataframe

    # Call get_balance_data_with_duplicates with the duplicate content
    balanced_data = get_balance_data_with_duplicates(duplicates, dataframe, pos_folder, neg_folder)

    # Recursively check for duplicates
    print("Recursive call to balance the data.")
    return recursive_data_balancing(balanced_data, column, pos_folder, neg_folder)

In [14]:
balanced_data = recursive_data_balancing(org_ID_removed_data, 'Content', pos_folder, neg_folder)

Repeated data entry/entries found in Content
Recursive call to balance the data.
No repeated data entry found.
No more duplicates found.


In [15]:
# check for null data
balanced_data.isnull().sum()

Sentiment    0
Rating       0
Content      0
dtype: int64

In [16]:
# double checking the total number of data entries
balanced_data['Sentiment'].value_counts()

Sentiment
pos    2000
neg    2000
Name: count, dtype: int64